In [50]:
import numpy as np
import pandas as pd

In [51]:
attempts_df = pd.read_csv("TextBanking_Efforts.xlsx - Attempts.csv")
contact_df = pd.read_csv("TextBanking_Efforts.xlsx - Contact.csv")

# clean -- values
for column in attempts_df.columns:
    attempts_df = attempts_df[(attempts_df[column] != "--")]
for column in contact_df.columns:
    contact_df = contact_df[(contact_df[column] != "--")]

In [52]:
attempts_df.columns

Index(['timestamp', 'message_direction', 'Voter ID'], dtype='object')

In [53]:
contact_df.columns

Index(['Question Sent', 'question_Response_type', 'response to question',
       'updated_at', 'Voter ID'],
      dtype='object')

In [59]:
# dictionary mapping {Question : {available response : count}}
question_response = {}
total_count = 0

# initialize questions
for question in contact_df["Question Sent"].unique():
    question_response[question] = {}

# initialize available responses per question
for row_idx in range(0, contact_df.shape[0]):
    # 'Question Sent' is col 0, 'response to question' is col 2
    question = contact_df.iloc[row_idx, 0]
    response = contact_df.iloc[row_idx, 2]
    if response not in question_response[question]:
        question_response[question][response] = 1
        total_count += 1
    else:
        question_response[question][response] += 1
        total_count += 1

print(contact_df["Voter ID"].unique().shape[0])
question_response

419


{'Planning to vote?': {'Yes': 200, 'No': 44},
 'Do you have the proper ID to vote?': {'Yes': 49, 'Unsure': 3, 'No': 2},
 'Texting correct person': {'Contacted': 142,
  'Refused': 53,
  'Wrong Number': 111,
  'Other Language': 1,
  'Spanish': 2},
 'Texting the Right Person': {'Wrong Number': 8}}

In [60]:
# Success at getting a response from reaching out, ie contact success rate
all_contacts_df = pd.DataFrame({"Voter ID": attempts_df["Voter ID"].unique()})
all_contacts_df["Contact Success"] = 0 # changed to 1 later if success
all_contacts_df = all_contacts_df.set_index('Voter ID')
all_contacts_df = all_contacts_df.sort_values(by=["Voter ID"])
contact_df = contact_df.sort_values(by=["Voter ID"])

In [61]:
### contact_df = contact_df.set_index('Voter ID')

# TODO: go through contact_df and set the voter ids to contact success
# Voter ID is column 4 in contact_df
prev_voterID = contact_df.iloc[0, 4]
success = False # list of questions this person was asked over text

for row_idx in range(0, contact_df.shape[0]):
    curr_voterID = contact_df.iloc[row_idx, 4]
    # if new person, analyze the boolean and reset boolean
    if curr_voterID != prev_voterID:
        # analyze previous voterID. If success, change in all_contacts_df
        if success:
            all_contacts_df.loc[prev_voterID, "Contact Success"] = 1
        success = False # default boolean to False
        # Consider any response other than (Yes, planning to vote), (Yes, have proper ID), (Contacted right person) to be unsuccessful
        question = contact_df.iloc[row_idx, 0]
        response = contact_df.iloc[row_idx, 2]
        print(curr_voterID, "!=", question, response)
        # ISSUE: PLANNING TO VOTE DOES NOT NECESSARILY COME FIRST
        if question == 'Planning to vote?' and response == 'Yes':
            success = True
        if question == 'Do you have the proper ID to vote?' and response != 'Yes':
            success = False
        if question == 'Texting correct person' and response != 'Contacted':
            success = False
        if question == 'Texting the Right Person' and response == 'Wrong Number':
            success = False

    if curr_voterID == prev_voterID:
        # questions_asked is column 0, response is column 2
        # Consider any response other than (Yes, planning to vote), (Yes, have proper ID), (Contacted right person) to be unsuccessful
        question = contact_df.iloc[row_idx, 0]
        response = contact_df.iloc[row_idx, 2]
        print(curr_voterID, "==", question, response)
        if question == 'Planning to vote?' and response == 'Yes':
            success = True
        if question == 'Do you have the proper ID to vote?' and response != 'Yes':
            success = False
        if question == 'Texting correct person' and response != 'Contacted':
            success = False
        if question == 'Texting the Right Person' and response == 'Wrong Number':
            success = False

    prev_voterID = curr_voterID

all_contacts_df

1031310768 == Planning to vote? Yes
1041564212 != Planning to vote? Yes
1041564212 == Do you have the proper ID to vote? Yes
1043427395 != Do you have the proper ID to vote? Yes
1043427395 == Texting correct person Contacted
1043427395 == Planning to vote? Yes
1061260847 != Planning to vote? Yes
1061260847 == Texting correct person Contacted
1090798474 != Planning to vote? Yes
1102265724 != Texting correct person Contacted
1102265724 == Planning to vote? No
1102280756 != Texting correct person Contacted
1102280756 == Planning to vote? Yes
1102288308 != Texting correct person Refused
1102288536 != Planning to vote? Yes
1102288536 == Texting correct person Contacted
1102298111 != Planning to vote? Yes
1102298111 == Texting correct person Contacted
1102311807 != Texting correct person Wrong Number
1102311807 == Planning to vote? Yes
1102319268 != Texting correct person Wrong Number
1102321583 != Planning to vote? Yes
1102321583 == Texting correct person Contacted
1102334131 != Texting cor

Contact Success
Voter ID                   
1000205011                0
1000433225                0
1000746850                0
1001614539                0
1004309873                0
...                     ...
2193193194                0
2193193564                0
2193194892                0
2193225791                0
2193229462                0

[10167 rows x 1 columns]

In [62]:
# sanity check to make sure all voter ids in contacts_df are also in all_contacts_df
count = 0
for voter_id in contact_df["Voter ID"].unique():
    if voter_id not in all_contacts_df.index:
        count += 1
count

0

In [63]:
# sanity check for previous block:
num_success = 0
for row_idx in range(0, all_contacts_df.shape[0]):
    if all_contacts_df.iloc[row_idx, 0] == 1:
        num_success += 1
num_success
# should be <200

195

In [64]:
# get final percentage of successful contacts
num_success / all_contacts_df.shape[0]

0.019179699026261435